<a href="https://colab.research.google.com/github/mkmagaya/Machine-Learning-Projects-Python-/blob/master/Q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec 
import random
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
import warnings;
warnings.filterwarnings('ignore')

In [ ]:
df_movies = pd.read_csv('movies.csv')
df_ratings = pd.read_csv('ratings.csv')

In [ ]:
df = pd.merge(df_movies,df_ratings)

In [ ]:
df.shape

(100836, 6)

In [ ]:
df.dropna(inplace=True)

In [ ]:
df['movieId']= df['movieId'].astype(str)
users = df["userId"].unique().tolist()
len(users)

610

In [ ]:
random.shuffle(users)
# extract 90% of user ID's
users_train = [users[i] for i in range(round(0.9*len(users)))]
# split data into train and validation set
train_df = df[df['userId'].isin(users_train)]
validation_df = df[~df['userId'].isin(users_train)]

In [ ]:
#list to capture watch history of the users
watch_train = []
# populate the list with the movie ID
for i in tqdm(users_train):
    temp = train_df[train_df["userId"] == i]["movieId"].tolist()
    watch_train.append(temp)

100%|██████████| 549/549 [00:00<00:00, 1048.92it/s]


In [ ]:
model = Word2Vec(window = 10, sg = 1, hs = 0,
                 negative = 10, 
                 alpha=0.03, min_alpha=0.0007,
                 seed = 14)
model.build_vocab(watch_train, progress_per=200)
model.train(watch_train, total_examples = model.corpus_count, 
            epochs=10, report_delay=1)

(769356, 874200)

In [ ]:
X = model[model.wv.vocab]
X.shape

(3313, 100)

In [ ]:
watch = train_df[["movieId", "title"]]
# remove duplicates
watch.drop_duplicates(inplace=True, subset='movieId', keep="last")
# create movie id and tittle dictionary
watch_dict = watch.groupby('movieId')['title'].apply(list).to_dict()

In [ ]:
def similar_watch(v, n = 6):
     
    # extract most similar movies for the input vector
    ms = model.similar_by_vector(v, topn= n+1)[1:]
     
    # extract name and similarity score of the similar movies
    new_ms = []
    for j in ms:
        pair = (watch_dict[j[0]][0], j[1])
        new_ms.append(pair)
         
    return new_ms

In [ ]:
print(watch_dict['307'])
similar_watch(model['307'])


['Three Colors: Blue (Trois couleurs: Bleu) (1993)']


[('Three Colors: Red (Trois couleurs: Rouge) (1994)', 0.988335371017456),
 ('Three Colors: White (Trzy kolory: Bialy) (1994)', 0.98509681224823),
 ('Shallow Grave (1994)', 0.9719000458717346),
 ('Secret of Roan Inish, The (1994)', 0.9622868895530701),
 ('Once Were Warriors (1994)', 0.9549619555473328),
 ('Swimming with Sharks (1995)', 0.9370958805084229)]

In [ ]:
print(watch_dict['1'])
similar_watch(model['1'])

['Toy Story (1995)']


[('Heat (1995)', 0.9217488169670105),
 ('Sense and Sensibility (1995)', 0.916376531124115),
 ('Jumanji (1995)', 0.911945104598999),
 ('Father of the Bride Part II (1995)', 0.9078569412231445),
 ('Leaving Las Vegas (1995)', 0.907021164894104),
 ('American President, The (1995)', 0.9068361520767212)]